In [6]:
import pandas as pd 
test_data = pd.read_csv('/workspaces/Wikistim-Summarization/00_source_data/uncleaned_text.csv')

In [7]:
test_data['full_article_text'][3]

'Vol.:(0123456789) 1 3 Quality of Life Research (2018) 27:2035–2044  https://doi.org/10.1007/s11136-018-1890-8 Long-term quality of\xa0life improvement for\xa0chronic intractable back  and\xa0leg pain patients using spinal cord stimulation: 12-month results  from\xa0the\xa0SENZA-RCT  Kasra\xa0Amirdelfan1\xa0· Cong\xa0Yu2\xa0· Matthew\xa0W.\xa0Doust3\xa0· Bradford\xa0E.\xa0Gliner4\xa0· Donna\xa0M.\xa0Morgan5\xa0· Leonardo\xa0Kapural6\xa0·  Ricardo\xa0Vallejo7\xa0· B.\xa0Todd\xa0Sitzman8\xa0· Thomas\xa0L.\xa0Yearwood9\xa0· Richard\xa0Bundschu10\xa0· Thomas\xa0Yang2\xa0·  Ramsin\xa0Benyamin7\xa0· Abram\xa0H.\xa0Burgher3\xa0· Elizabeth\xa0S.\xa0Brooks4\xa0· Ashley\xa0A.\xa0Powell4\xa0· Jeyakumar\xa0Subbaroyan4 Accepted: 22 May 2018 / Published online: 1 June 2018  © The Author(s) 2018 Abstract Purpose Chronic axial low-back pain is a debilitating disorder that impacts all aspects of an afflicted individual’s life. Effec- tive, durable treatments have historically been elusive. Intervention

In [2]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/long-t5-local-base"
model_dir = "./saved_model_long_t5_local_base"

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Try to load the model from the cache or from the specified directory
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, cache_dir=model_dir)
except OSError:
    # If the model is not found in cache or directory, download and save it
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, cache_dir=model_dir)
    model.save_pretrained(model_dir)

Some weights of LongT5ForConditionalGeneration were not initialized from the model checkpoint at google/long-t5-local-base and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from transformers import pipeline 

text_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, min_length=40, max_length=100)

2023-10-11 19:17:32.322763: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 19:17:32.360307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-11 19:17:32.360351: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-11 19:17:32.360387: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-11 19:17:32.369897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [8]:
my_test = test_data['full_article_text'][3]


In [9]:

# Define the prefix
prefix = "summarize:"
max_context_length = 511

# Tokenize the input text
tokenized_input = tokenizer(my_test, truncation=False)
tokenized_prefix = tokenizer(prefix, truncation=False)

prefix_length = len(tokenized_prefix["input_ids"])

max_chunk_length = max_context_length - prefix_length



chunks = []

# Start with the full text
current_chunk = []

for i in range(len(tokenized_input["input_ids"])):
    current_chunk.append(tokenized_input["input_ids"][i])

    # Check if the current chunk size exceeds the max_chunk_size
    if len(current_chunk) >= max_chunk_length:
        chunks.append(current_chunk[:])  # Make a copy of the current chunk
        current_chunk = []  # Reset the current chunk

# If there's any remaining tokens in the current chunk, add it to the list of chunks
if len(current_chunk) > 0:
    chunks.append(current_chunk)

# Print the number of chunks and tokens in each chunk
print("Number of chunks:", len(chunks))
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1} - Number of tokens:", len(chunk))



Number of chunks: 24
Chunk 1 - Number of tokens: 508
Chunk 2 - Number of tokens: 508
Chunk 3 - Number of tokens: 508
Chunk 4 - Number of tokens: 508
Chunk 5 - Number of tokens: 508
Chunk 6 - Number of tokens: 508
Chunk 7 - Number of tokens: 508
Chunk 8 - Number of tokens: 508
Chunk 9 - Number of tokens: 508
Chunk 10 - Number of tokens: 508
Chunk 11 - Number of tokens: 508
Chunk 12 - Number of tokens: 508
Chunk 13 - Number of tokens: 508
Chunk 14 - Number of tokens: 508
Chunk 15 - Number of tokens: 508
Chunk 16 - Number of tokens: 508
Chunk 17 - Number of tokens: 508
Chunk 18 - Number of tokens: 508
Chunk 19 - Number of tokens: 508
Chunk 20 - Number of tokens: 508
Chunk 21 - Number of tokens: 508
Chunk 22 - Number of tokens: 508
Chunk 23 - Number of tokens: 508
Chunk 24 - Number of tokens: 43


In [10]:
import unittest

def check_context_lengths(): 
    for idx, chunk in enumerate(chunks, 1): 
        if len(tokenizer(tokenizer.decode(tokenized_prefix["input_ids"]) + tokenizer.decode(chunks[0]))["input_ids"]) > 512:
            raise ValueError(f"Chunk {idx - 1} is too long")
        else: 
            pass

    return True

check_context_lengths()

True

In [11]:
for chunk in range(0, len(chunks)):
    print(f"Chunk number: {chunk}")
    decoded_text = tokenizer.decode(tokenized_prefix["input_ids"]) + tokenizer.decode(chunks[chunk])
    print(text_generator(decoded_text))
    print('-----------------------------------')

Chunk number: 0


/home/codespace/.local/lib/python3.10/site-packages/transformers/models/longt5/modeling_longt5.py:147: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 128, 1], which does not match the required output shape [1, 4, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  local_attention_mask = torch.logical_and(_blocked_attention_mask, _3blocked_attention_mask)
/home/codespace/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:859: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[{'generated_text': 'extraordinaire extraordinairemobile extraordinaireMar cazarefatasigurareastehenden vorbe Published extraordinaire dificil élevé retro quarterback strictly retro retro retro graph feed extraordinaire dificil Restaurant élevéstehendenstehendenstehendenstehendenAstăzi imitation quarterback Restaurant Published dificil Restaurant solicitatfatmobile definitelyhafen].].].154mobile definitely being BrookOuest retro graph citoyencopy retroacești dificil deleted extraordinaire dificil Restaurant RestaurantShare Au albastru élevéstehenden TOP sterlingmobilestehenden TOP sterling Browse154 Browse154 graphShare154anno customers].154autres dificil solicitatfatasigurarea Restaurantshri Democratsespecially Join warrantiesANI Democratsespecially'}]
-----------------------------------
Chunk number: 1
[{'generated_text': 'extraordinaire extraordinaire extraordinaire milieuVO reconnufat Beckfat extraordinaire dificil élevé extraordinaire élevé élevé élevéstehenden élevéVOmobile extra

/home/codespace/.local/lib/python3.10/site-packages/transformers/models/longt5/modeling_longt5.py:147: UserWarning: An output with one or more elements was resized since it had shape [1, 5, 128, 1], which does not match the required output shape [1, 5, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  local_attention_mask = torch.logical_and(_blocked_attention_mask, _3blocked_attention_mask)


[{'generated_text': 'extraordinaire vessels extraordinaire wrestle strictly Au extraordinairegelangtgelangt strictly strictly strictly strictly ladies citoyencopy 3:1track Restaurant RestaurantShare extraordinaire MustDIS emb extraordinaire Must].kauffix154OLD extraordinaire154 Published extraordinaire154 Published Backup154 Published Backup154154 extraordinaire154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154154calorieOLD solicitat țară'}]
-----------------------------------
Chunk number: 4
[{'generated_text': 'ethanol extraordinaire extraordinaire extraordinairewin Au extraordinairegelangt imitation brûl Tempegelangt Regi accept élevé extraordinairestehenden Rebecca graph Rebecca graph perfekt opponent clinician extraordinaire dificil Rebecca paperwork Religiousgelangt beingresident extraordinaire dificilținut being Restaurant beingresident dificil entertaining154154 extraordinaire dific

/home/codespace/.local/lib/python3.10/site-packages/transformers/models/longt5/modeling_longt5.py:147: UserWarning: An output with one or more elements was resized since it had shape [1, 1, 128, 1], which does not match the required output shape [1, 1, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  local_attention_mask = torch.logical_and(_blocked_attention_mask, _3blocked_attention_mask)


[{'generated_text': 'extraordinaire extraordinaire milieu Democrats paperwork détour Browse Au aceasta găsi 14. paperwork Democrats oasisinaugurationÜbertragung imitation élevé extraordinaire élevé extraordinaireuben extraordinaireubenubenwin Au falling Restaurant Liegeținut Browse imitation Restaurant opponent extraordinaire dificil élevé dificil Restaurant Democrats élevé dificil Restaurant Democrats Democrats Democrats élevé dificil paperwork paperworkautresținut Restaurant Democrats Regifat forfeit imitation paperwork paperwork Restaurantautres oxygen élevé dificil oasis paperwork Regiapprentissage Religious élevé Restaurant woodland Religiousținut oasis oasis reset élevé dificil oasis oasisSullivan Democrats extraordinaire dificil Virginia oasisținut élevé dificil oasis oasis definitely Published extraordinaire dificil oasis'}]
-----------------------------------
